# 프로젝트: ResNet Ablation Study

In [17]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

# from keras import models, layers
# from keras import Input
# from keras.models import Model, load_model
# from keras.preprocessing.image import ImageDataGenerator
# from keras import optimizers, initializers, regularizers, metrics
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

import tensorflow_datasets as tfds
tf.config.list_physical_devices('GPU')

In [11]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [13]:
# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요. 
print(ds_info.features)

In [ ]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

In [ ]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [ ]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

## ResNet 기본 블록 구성하기

In [26]:
# function for building VGG Block

# block_num : 블록 이름 붙이기 위해
# is_50 ResNet-50인지 구분하기 위해
def build_resnet_block(input_layer,
                    num_cnn=3, 
                    channel=64,
                    block_num=1,
                    is_50 = False
                   ):    

    # 입력 레이어
    x = input_layer
    # shortcut 생성
    x_shortcut = x

    # ResNet-50인 경우
    if is_50:
        for cnn_num in range(num_cnn):
            print('========== ResNet-50 Block_{}_{} =============='.format(block_num,cnn_num))
            
            # 첫 번째 블록을 제외한 나머지 블록에서 첫 번째 conv layer 는 stride 가 (2,2) 이어야 한다.
            # 그리고 첫 번째 블록들은 다 마지막에 shortcut 모양맞춰주기를 해야한다.

            strides = (1,1)                  
            # 첫 번째 블록의 첫 번째 conv layer만 stides (1,1)이고 나머지 블록의 첫 번째 conv layer 는 (2,2)임.
            if cnn_num == 0 and block_num != 0:
                strides = (2,2)                                

            x = Conv2D(filters = channel, kernel_size = (1, 1), strides= strides, padding='valid', name=f'block_{block_num}_1x1_{cnn_num}_1')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_1')(x)
            x = Activation('relu')(x)
            
            x = Conv2D(filters = channel, kernel_size = (3,3), strides= (1,1), padding='same',  name=f'block_{block_num}_conv_{cnn_num}')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_2')(x)
            x = Activation('relu')(x)

            x = Conv2D(filters = channel*4, kernel_size = (1, 1), strides= (1,1), padding='valid', name=f'block_{block_num}_1x1_{cnn_num}_2')(x)                
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_3')(x)
            
            # 각 블록의 첫 번째 conv layer 인 경우, shortcut 모양 맞춰주기
            if cnn_num == 0:

                x_shortcut = Conv2D(channel*4, (1, 1), strides=strides, padding='valid')(x_shortcut)            
                x_shortcut = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_shortcut')(x_shortcut)
    
            x = Add()([x, x_shortcut])
            x = Activation('relu')(x)
                
            x_shortcut = x
                
    #  ResNet-34인 경우
    else:
        for cnn_num in range(num_cnn):
            print('========== ResNet-34 Block_{}_{} =============='.format(block_num,cnn_num))

            strides = (1,1)                  
            # 첫 번째 블록의 첫 번째 conv layer만 stides (1,1)이고 나머지 블록의 첫 번째 conv layer 는(2,2)임.
            if cnn_num == 0 and block_num != 0:
                strides = (2,2)  
                     
            x = Conv2D(filters = channel, kernel_size = (3, 3), strides= strides, padding='same', name=f'block_{block_num}_conv_{cnn_num}_1')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_1')(x)
            x = Activation('relu')(x)

            x = Conv2D(filters = channel, kernel_size = (3,3), strides= (1,1), padding='same',  name=f'block_{block_num}_conv_{cnn_num}_2')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_2')(x)

            # 각 블록의 첫 번째 conv layer 인 경우, shortcut 모양 맞춰주기
            if cnn_num == 0:

                x_shortcut = Conv2D(channel, (1, 1), strides=strides, padding='valid')(x_shortcut)            
                x_shortcut = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_shortcut')(x_shortcut)                              

            x = keras.layers.Add()([x,x_shortcut])
            x = keras.layers.Activation('relu')(x)

            x_shortcut = x

    return x

## ResNet-34, ResNet-50 Complete Model

In [32]:
def build_resnet(input_shape=(32,32,3),is_50 = False, num_classes=1000):

    num_cnn_list = [3,4,6,3]
    channel_list = [64,128,256,512]
    num_classes = num_classes

    input_layer = Input(shape=input_shape, dtype='float32', name='input')  # input layer를 만들어둡니다.
    
    x = input_layer
    
    # conv_1 layer
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(
            filters=64,
            kernel_size=(7,7),
            strides = (2,2),
            name ='conv1'
        )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)    

    # conv_2 layer maxpool 먼저
    x = MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            name=f'conv2_max_pool'
        )(x) 
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = build_resnet_block(
            input_layer = x,
            num_cnn=num_cnn, 
            channel=channel,            
            block_num=i,
            is_50 = is_50
        )

    x = GlobalAveragePooling2D(name=f'average_pool')(x)    
    output = keras.layers.Dense(num_classes, activation='softmax', name='fc')(x)    

    model = keras.Model(
        inputs = input_layer,
        outputs = output
    )

    return model


In [33]:
resnet_34 = build_resnet(input_shape=(32,32,3),is_50 = False)
resnet_34.summary()

[0][0]           
__________________________________________________________________________________________________
block_1_0_bn_shortcut (BatchNor (None, 4, 4, 128)    512         conv2d_17[0][0]                  
__________________________________________________________________________________________________
add_59 (Add)                    (None, 4, 4, 128)    0           block_1_0_bn_2[0][0]             
                                                                 block_1_0_bn_shortcut[0][0]      
__________________________________________________________________________________________________
activation_144 (Activation)     (None, 4, 4, 128)    0           add_59[0][0]                     
__________________________________________________________________________________________________
block_1_conv_1_1 (Conv2D)       (None, 4, 4, 128)    147584      activation_144[0][0]             
___________________________________________________________________________________________

In [34]:
resnet_50 = build_resnet(input_shape=(32,32,3),is_50 = True)
resnet_50.summary()

][0]             
__________________________________________________________________________________________________
block_2_0_bn_1 (BatchNormalizat (None, 2, 2, 256)    1024        block_2_1x1_0_1[0][0]            
__________________________________________________________________________________________________
activation_191 (Activation)     (None, 2, 2, 256)    0           block_2_0_bn_1[0][0]             
__________________________________________________________________________________________________
block_2_conv_0 (Conv2D)         (None, 2, 2, 256)    590080      activation_191[0][0]             
__________________________________________________________________________________________________
block_2_0_bn_2 (BatchNormalizat (None, 2, 2, 256)    1024        block_2_conv_0[0][0]             
__________________________________________________________________________________________________
activation_192 (Activation)     (None, 2, 2, 256)    0           block_2_0_bn_2[0][0]      

## 일반 네트워크(plain network) 만들기

In [37]:
# function for building VGG Block

# block_num : 블록 이름 붙이기 위해
# is_50 Plane-50인지 구분하기 위해
def build_plane_block(input_layer,
                    num_cnn=3, 
                    channel=64,
                    block_num=1,
                    is_50 = False
                   ):    

    # 입력 레이어
    x = input_layer

    # Plane-50인 경우
    if is_50:
        for cnn_num in range(num_cnn):
            print('========== Plane-50 Block_{}_{} =============='.format(block_num,cnn_num))
            
            # 첫 번째 블록을 제외한 나머지 블록에서 첫 번째 conv layer 는 stride 가 (2,2) 이어야 한다.
            # 그리고 첫 번째 블록들은 다 마지막에 shortcut 모양맞춰주기를 해야한다.

            strides = (1,1)                  
            # 첫 번째 블록의 첫 번째 conv layer만 stides (1,1)이고 나머지 블록의 첫 번째 conv layer 는 (2,2)임.
            if cnn_num == 0 and block_num != 0:
                strides = (2,2)                                

            x = Conv2D(filters = channel, kernel_size = (1, 1), strides= strides, padding='valid', name=f'block_{block_num}_1x1_{cnn_num}_1')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_1')(x)
            x = Activation('relu')(x)
            
            x = Conv2D(filters = channel, kernel_size = (3,3), strides= (1,1), padding='same',  name=f'block_{block_num}_conv_{cnn_num}')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_2')(x)
            x = Activation('relu')(x)

            x = Conv2D(filters = channel*4, kernel_size = (1, 1), strides= (1,1), padding='valid', name=f'block_{block_num}_1x1_{cnn_num}_2')(x)                
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_3')(x)
            x = Activation('relu')(x)                       
                
    # Plane-34인 경우
    else:
        for cnn_num in range(num_cnn):
            print('========== Plane-34 Block_{}_{} =============='.format(block_num,cnn_num))

            strides = (1,1)                  
            # 첫 번째 블록의 첫 번째 conv layer만 stides (1,1)이고 나머지 블록의 첫 번째 conv layer 는(2,2)임.
            if cnn_num == 0 and block_num != 0:
                strides = (2,2)  
                     
            x = Conv2D(filters = channel, kernel_size = (3, 3), strides= strides, padding='same', name=f'block_{block_num}_conv_{cnn_num}_1')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_1')(x)
            x = Activation('relu')(x)

            x = Conv2D(filters = channel, kernel_size = (3,3), strides= (1,1), padding='same',  name=f'block_{block_num}_conv_{cnn_num}_2')(x)
            x = BatchNormalization(name=f'block_{block_num}_{cnn_num}_bn_2')(x)
            x = keras.layers.Activation('relu')(x)
            
    return x

In [38]:
def build_plane(input_shape=(32,32,3),is_50 = False, num_classes=1000):

    num_cnn_list = [3,4,6,3]
    channel_list = [64,128,256,512]
    num_classes = num_classes

    input_layer = Input(shape=input_shape, dtype='float32', name='input')  # input layer를 만들어둡니다.
    
    x = input_layer
    
    # conv_1 layer
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(
            filters=64,
            kernel_size=(7,7),
            strides = (2,2),
            name ='conv1'
        )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)    

    # conv_2 layer maxpool 먼저
    x = MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            name=f'conv2_max_pool'
        )(x) 
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = build_resnet_block(
            input_layer = x,
            num_cnn=num_cnn, 
            channel=channel,            
            block_num=i,
            is_50 = is_50
        )

    x = GlobalAveragePooling2D(name=f'average_pool')(x)    
    output = keras.layers.Dense(num_classes, activation='softmax', name='fc')(x)    

    model = keras.Model(
        inputs = input_layer,
        outputs = output
    )

    return model


In [39]:
plane_34 = build_plane(input_shape=(32,32,3),is_50 = False)
plane_34.summary()

[0][0]           
__________________________________________________________________________________________________
block_1_0_bn_shortcut (BatchNor (None, 4, 4, 128)    512         conv2d_25[0][0]                  
__________________________________________________________________________________________________
add_91 (Add)                    (None, 4, 4, 128)    0           block_1_0_bn_2[0][0]             
                                                                 block_1_0_bn_shortcut[0][0]      
__________________________________________________________________________________________________
activation_226 (Activation)     (None, 4, 4, 128)    0           add_91[0][0]                     
__________________________________________________________________________________________________
block_1_conv_1_1 (Conv2D)       (None, 4, 4, 128)    147584      activation_226[0][0]             
___________________________________________________________________________________________

In [40]:
plane_50 = build_plane(input_shape=(32,32,3),is_50 = True)
plane_50.summary()

][0]             
__________________________________________________________________________________________________
block_2_0_bn_1 (BatchNormalizat (None, 2, 2, 256)    1024        block_2_1x1_0_1[0][0]            
__________________________________________________________________________________________________
activation_273 (Activation)     (None, 2, 2, 256)    0           block_2_0_bn_1[0][0]             
__________________________________________________________________________________________________
block_2_conv_0 (Conv2D)         (None, 2, 2, 256)    590080      activation_273[0][0]             
__________________________________________________________________________________________________
block_2_0_bn_2 (BatchNormalizat (None, 2, 2, 256)    1024        block_2_conv_0[0][0]             
__________________________________________________________________________________________________
activation_274 (Activation)     (None, 2, 2, 256)    0           block_2_0_bn_2[0][0]      

## ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34 _ input_shape : (32,32,3)


### input_shape (224,224,3) 으로 비교해보기 : cat_vs_dogs 로 비교해보기